In [ ]:
# Data processing
import iris
import numpy as np
import xarray as xr
import timeit
import dateutil.parser
import warnings
warnings.filterwarnings('ignore', module='iris')
from iris.experimental import stratify
from iris.analysis import trajectory
from pathlib import Path
from datetime import datetime, timedelta
# Visualization
import matplotlib.pyplot as plt
## Scientific parameters
M_air = 28.97 # molar mass of dry air [kg]
# Coefficients to convert mass mixing ratio to volume mixing ratio
coeff_c2h6 = (M_air/30.0690)*1e12
coeff_c3h8 = (M_air/44.0956)*1e12
coeff_nc4h10 = (M_air/58.1222)*1e12
coeff_ic4h10 = (M_air/58.1222)*1e12
coeff_nc5h12 = (M_air/72.1488)*1e12
coeff_ic5h12 = (M_air/72.1488)*1e12
coeff_meono2 = (M_air/77.0394)*1e12
coeff_sbuono2 = (M_air/119.1192)*1e12
coeff_n2peono2 = (M_air/133.1457)*1e12
coeff_n3peono2 = (M_air/133.1457)*1e12
coeff_ipeono2 = (M_air/133.1457)*1e12

#### Read data

In [ ]:
path_to_atom = Path('../../external_data/ATom/nc/data')
# Select flights
flist_atom_flights = ['MER-WAS_DC8_20170218_R8.nc', 'MER-WAS_DC8_20170219_R7.nc', 'MER-WAS_DC8_20170221_R8.nc']
fpaths_atom_flights = [path_to_atom / i for i in flist_atom_flights]
# Read ATom data
atom_dsinf = xr.open_mfdataset(fpaths_atom_flights, decode_times=True)
atom_dsmms = xr.open_mfdataset(fpaths_atom_flights, group='MMS', decode_cf=True)
atom_dswas = xr.open_mfdataset(fpaths_atom_flights, group='WAS', decode_cf=True)

In [ ]:
# Choose UKCA run parameters
ukca_run_name = 'ba897'; ukca_run_freq = 'T1H'; ukca_run_mnth = '201702'; lvs = 'thlevs'

path_to_ukca = Path('../data') / ukca_run_name
fname_ukca_run = f'{ukca_run_name}_{ukca_run_freq}_{ukca_run_mnth}_{lvs}.nc'
# Read UKCA data
_cb_c2h6 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'C2H6 MASS MIXING RATIO AFTER TSTEP')*coeff_c2h6
_cb_c3h8 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'C3H8 MASS MIXING RATIO AFTER TSTEP')*coeff_c3h8
_cb_nc4h10 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'n-C4H10 MASS MIXING RATIO AFTER TS')*coeff_nc4h10
_cb_ic4h10 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'i-C4H10 MASS MIXING RATIO AFTER TS')*coeff_ic4h10
_cb_nc5h12 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'n-C5H12 MASS MIXING RATIO AFTER TS')*coeff_nc5h12
_cb_ic5h12 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'i-C5H12 MASS MIXING RATIO AFTER TS')*coeff_ic5h12
_cb_meono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'MeONO2 MASS MIXING RATIO AFTER TSTEP')*coeff_meono2
_cb_sbuono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 's-BuONO2 MASS MIXING RATIO AFTER TS')*coeff_sbuono2
_cb_n2peono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'n-2PeONO2 MASS MIXING RATIO AFTER TS')*coeff_n2peono2
_cb_n3peono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'n-3PeONO2 MASS MIXING RATIO AFTER TS')*coeff_n3peono2
_cb_ipeono2 = iris.load_cube(str(path_to_ukca / fname_ukca_run), 'i-PeONO2 MASS MIXING RATIO AFTER TS')*coeff_ipeono2
# Extract needed longitude range (ba897 run extends further east than ba083 to allow comparison with ACSIS flights)
cb_c2h6 = _cb_c2h6.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
cb_c3h8 = _cb_c3h8.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
cb_nc4h10 = _cb_nc4h10.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
cb_ic4h10 = _cb_ic4h10.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
cb_nc5h12 = _cb_nc5h12.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
cb_ic5h12 = _cb_ic5h12.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
cb_meono2 = _cb_meono2.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
cb_sbuono2 = _cb_sbuono2.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
cb_n2peono2 = _cb_n2peono2.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
cb_n3peono2 = _cb_n3peono2.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
cb_ipeono2 = _cb_ipeono2.extract(iris.Constraint(longitude=lambda cell: 199.6875 <= cell <= 351.5625))
# Load altitude coordinate of UKCA's global domain
_alt_coord = iris.load_cube(str(Path('../data')/'ukca_coords_n96e_marcus_180705.nc'),'mass_fraction_of_ozone_in_air').coord('altitude')

In [ ]:
# Extract datetimes from ATom and UKCA data
atom_datetime = atom_dsinf.time.values.astype('<M8[us]').astype(datetime)
atom_date_strt = atom_datetime[0]
atom_date_stop = atom_datetime[-1]
ukca_days_since = dateutil.parser.parse(str(cb_nc4h10.coord('t').units)[11:])
_ukca_datetime = []
for i, ii in enumerate(list(cb_nc4h10.coord('t').points)):
    _ukca_datetime.append(ukca_days_since + timedelta(days=float(f'{ii:1.3f}'))) # UKCA's 'days since 1960-09-01' format is 1.3f
ukca_datetime = np.array(_ukca_datetime, dtype=datetime)
# Create a common timestamp/flight merge identifier for trajectories
compared_common_id = f'{atom_date_strt.strftime("%y%m%d")}_{atom_date_stop.strftime("%y%m%d")}_{ukca_datetime[0].strftime("%y%m")}_merge_zlvs'
print(compared_common_id)

#### Process data
##### 1. Relevel model data from hybrid height to absolute height 

In [ ]:
# Manually select a part of global UKCA domain of altitudes to match current UKCA run
alt_coord = _alt_coord[0:44,84:144,106:-4]
# Add altitude coordinate to the cube with model data
cb_c2h6.add_aux_coord(alt_coord, data_dims=[1,2,3])
cb_c3h8.add_aux_coord(alt_coord, data_dims=[1,2,3])
cb_nc4h10.add_aux_coord(alt_coord, data_dims=[1,2,3])
cb_ic4h10.add_aux_coord(alt_coord, data_dims=[1,2,3])
cb_nc5h12.add_aux_coord(alt_coord, data_dims=[1,2,3])
cb_ic5h12.add_aux_coord(alt_coord, data_dims=[1,2,3])
cb_meono2.add_aux_coord(alt_coord, data_dims=[1,2,3])
cb_sbuono2.add_aux_coord(alt_coord, data_dims=[1,2,3])
cb_n2peono2.add_aux_coord(alt_coord, data_dims=[1,2,3])
cb_n3peono2.add_aux_coord(alt_coord, data_dims=[1,2,3])
cb_ipeono2.add_aux_coord(alt_coord, data_dims=[1,2,3])

Warning: compute one variable at a time to avoid 'frozen' kernel

In [ ]:
# Relevel model data from hybrid height to target levels
tgt_levels = np.arange(0, 14000, 100) # [meters]
cbr_c2h6 = stratify.relevel(cb_c2h6, alt_coord, tgt_levels, axis='Hybrid height')
cbr_c3h8 = stratify.relevel(cb_c3h8, alt_coord, tgt_levels, axis='Hybrid height')
cbr_nc4h10 = stratify.relevel(cb_nc4h10, alt_coord, tgt_levels, axis='Hybrid height')
cbr_ic4h10 = stratify.relevel(cb_ic4h10, alt_coord, tgt_levels, axis='Hybrid height')
cbr_nc5h12 = stratify.relevel(cb_nc5h12, alt_coord, tgt_levels, axis='Hybrid height')
cbr_ic5h12 = stratify.relevel(cb_ic5h12, alt_coord, tgt_levels, axis='Hybrid height')
cbr_meono2 = stratify.relevel(cb_meono2, alt_coord, tgt_levels, axis='Hybrid height')
cbr_sbuono2 = stratify.relevel(cb_sbuono2, alt_coord, tgt_levels, axis='Hybrid height')
cbr_n2peono2 = stratify.relevel(cb_n2peono2, alt_coord, tgt_levels, axis='Hybrid height')
cbr_n3peono2 = stratify.relevel(cb_n3peono2, alt_coord, tgt_levels, axis='Hybrid height')
cbr_ipeono2 = stratify.relevel(cb_ipeono2, alt_coord, tgt_levels, axis='Hybrid height')

##### 2. Calculate trajectories

In [ ]:
# Reconstruct a flight trajectory from ATom geospacial and datetime data
sample_time = np.array([(i - ukca_days_since).total_seconds() / 86400. for i in atom_datetime])
sample_lats = atom_dsmms.G_LAT.data 
sample_lons = atom_dsmms.G_LONG.data+360
sample_alts = atom_dsmms.G_ALT.data
sample_time_lats_lons_alts = [('t', sample_time), ('latitude', sample_lats), ('longitude', sample_lons), ('altitude', sample_alts)]

In [ ]:
start_time = timeit.default_timer()
# Interpolate releveled model data to flight trajectory
traj_c2h6 = trajectory.interpolate(cbr_c2h6, sample_time_lats_lons_alts)
traj_c3h8 = trajectory.interpolate(cbr_c3h8, sample_time_lats_lons_alts)
traj_nc4h10 = trajectory.interpolate(cbr_nc4h10, sample_time_lats_lons_alts)
traj_ic4h10 = trajectory.interpolate(cbr_ic4h10, sample_time_lats_lons_alts)
traj_nc5h12 = trajectory.interpolate(cbr_nc5h12, sample_time_lats_lons_alts)
traj_ic5h12 = trajectory.interpolate(cbr_ic5h12, sample_time_lats_lons_alts)
traj_meono2 = trajectory.interpolate(cbr_meono2, sample_time_lats_lons_alts)
traj_sbuono2 = trajectory.interpolate(cbr_sbuono2, sample_time_lats_lons_alts)
traj_n2peono2 = trajectory.interpolate(cbr_n2peono2, sample_time_lats_lons_alts)
traj_n3peono2 = trajectory.interpolate(cbr_n3peono2, sample_time_lats_lons_alts)
traj_ipeono2 = trajectory.interpolate(cbr_ipeono2, sample_time_lats_lons_alts)
elapsed = timeit.default_timer() - start_time
elapsed/60

##### 3. Extract cross sections

In [ ]:
# Extract a cross section from model data along flight track
sample_time_lats_lons = [('t', sample_time), ('latitude', sample_lats), ('longitude', sample_lons)]
crsc_c2h6 = trajectory.interpolate(cbr_c2h6, sample_time_lats_lons)
crsc_c3h8 = trajectory.interpolate(cbr_c3h8, sample_time_lats_lons)
crsc_nc4h10 = trajectory.interpolate(cbr_nc4h10, sample_time_lats_lons)
crsc_ic4h10 = trajectory.interpolate(cbr_ic4h10, sample_time_lats_lons)
crsc_nc5h12 = trajectory.interpolate(cbr_nc5h12, sample_time_lats_lons)
crsc_ic5h12 = trajectory.interpolate(cbr_ic5h12, sample_time_lats_lons)
crsc_meono2 = trajectory.interpolate(cbr_meono2, sample_time_lats_lons)
crsc_sbuono2 = trajectory.interpolate(cbr_sbuono2, sample_time_lats_lons)
crsc_n2peono2 = trajectory.interpolate(cbr_n2peono2, sample_time_lats_lons)
crsc_n3peono2 = trajectory.interpolate(cbr_n3peono2, sample_time_lats_lons)
crsc_ipeono2 = trajectory.interpolate(cbr_ipeono2, sample_time_lats_lons)

#### Save processed data

In [ ]:
path_to_prcd = Path('../processed') / ukca_run_name

In [ ]:
# Save trajectories to .nc
iris.save(traj_c2h6, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_c2h6.nc')
iris.save(traj_c3h8, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_c3h8.nc')
iris.save(traj_nc4h10, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_nc4h10.nc')
iris.save(traj_ic4h10, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_ic4h10.nc')
iris.save(traj_nc5h12, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_nc5h12.nc')
iris.save(traj_ic5h12, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_ic5h12.nc')
iris.save(traj_meono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_meono2.nc')
iris.save(traj_sbuono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_sbuono2.nc')
iris.save(traj_n2peono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_n2peono2.nc')
iris.save(traj_n3peono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_n3peono2.nc')
iris.save(traj_ipeono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_traj_ipeono2.nc')

In [ ]:
# Save cross sections to .nc
iris.save(crsc_c2h6, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_c2h6.nc')
iris.save(crsc_c3h8, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_c3h8.nc')
iris.save(crsc_nc4h10, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_nc4h10.nc')
iris.save(crsc_ic4h10, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_ic4h10.nc')
iris.save(crsc_nc5h12, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_nc5h12.nc')
iris.save(crsc_ic5h12, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_ic5h12.nc')
iris.save(crsc_meono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_meono2.nc')
iris.save(crsc_sbuono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_sbuono2.nc')
iris.save(crsc_n2peono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_n2peono2.nc')
iris.save(crsc_n3peono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_n3peono2.nc')
iris.save(crsc_ipeono2, str(path_to_prcd)+f'/{ukca_run_name}_{compared_common_id}_crsc_ipeono2.nc')